In [1]:
from src.settings import FUNCTIONAL_DATA_DIR
from pathlib import Path
from geopandas import GeoDataFrame
import pandas as pd
from keplergl import KeplerGl
import plotly.express as px
import pickle as pkl

TOP_LEVEL_OSM_TAGS = [
    "aerialway",
    "aeroway",
    "amenity",
    "barrier",
    "boundary",
    "building",
    "craft",
    "emergency",
    "geological",
    "healthcare",
    "highway",
    "historic",
    "landuse",
    "leisure",
    "man_made",
    "military",
    "natural",
    "office",
    "place",
    "power",
    "public_transport",
    "railway",
    "route",
    "shop",
    "sport",
    "telecom",
    "tourism",
    "water",
    "waterway",
]

def load_city_tag_raw(city: str, tag: str) -> GeoDataFrame:
    path = Path(FUNCTIONAL_DATA_DIR).joinpath(city, f"{tag}.pkl")
    df = pd.read_pickle(path)
    gdf = GeoDataFrame(df)
    return gdf

def load_city_tag_counts(city: str, tag: str) -> pd.DataFrame:
    path = Path(FUNCTIONAL_DATA_DIR).joinpath(city, "counts", f"{tag}.pkl")
    df = pd.read_pickle(path)
    return df


In [2]:
def load_config(tag: str):
    with open(f"k_{tag}_c.pkl", "rb") as f:
        return pkl.load(f)

In [3]:
city = "Wrocław, PL"
results = []
for tag in TOP_LEVEL_OSM_TAGS:
    try:
        df = load_city_tag_raw(city, tag)
        tag_count = df[tag].count()
        results.append([tag, tag_count])
    except FileNotFoundError:
        print(f"File not found for city {city}, tag: {tag}")

results = pd.DataFrame(results, columns=['tag', 'count'])
results

File not found for city Wrocław, PL, tag: geological


,tag,count
0,aerialway,7
1,aeroway,134
2,amenity,14834
3,barrier,20156
4,boundary,42
5,building,155658
6,craft,103
7,emergency,2356
8,healthcare,319
9,highway,97310


In [19]:
px.bar(results, x='tag', y='count')

In [5]:
building_df = load_city_tag_raw(city, "building")
building_counts = building_df[['osmid', 'building']].groupby('building').count().rename(columns={"osmid": "count"})
building_counts

,count
building,
allotment_house,176
apartments,3220
barn,2
basilica,1
bunker,23
...,...
wall,1
warehouse,93
waste,1


In [20]:
px.bar(building_counts.sort_values('count', ascending=False).head(15))

In [24]:
amenity_df = load_city_tag_raw(city, "amenity")
amenity_counts = amenity_df[['osmid', 'amenity']].groupby('amenity').count().rename(columns={"osmid": "count"})
amenity_counts.reset_index()

,amenity,count
0,animal_breeding,1
1,animal_shelter,1
2,arts_centre,23
3,atm,336
4,baby_hatch,1
...,...,...
106,vending_machine,176
107,veterinary,48
108,waste_basket,443
109,waste_disposal,392


In [25]:
px.bar(amenity_counts.sort_values('count', ascending=False).head(15))

In [9]:
leisure_df = load_city_tag_raw(city, "leisure")
leisure_counts = leisure_df[['osmid', 'leisure']].groupby('leisure').count().rename(columns={"osmid": "count"})
leisure_counts

,count
leisure,
adult_gaming_centre,1
amusement_arcade,4
bandstand,2
bleachers,9
bowling_alley,3
common,34
dance,5
dog_park,13
escape_game,5


In [10]:
px.bar(leisure_counts)

In [11]:
natural_df = load_city_tag_raw(city, "natural")
natural_counts = natural_df[['osmid', 'natural']].groupby('natural').count().rename(columns={"osmid": "count"})
natural_counts

,count
natural,
bare_rock,2
beach,15
birds_nest,11
cliff,14
fossil,1
grassland,732
heath,86
hill,9
hillock,3


In [12]:
px.bar(natural_counts)

In [26]:
natural = load_city_tag_raw(city, "natural")
k_natural = KeplerGl(data={"natural": natural}, height=900)
k_natural

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'natural':                unique_id      osmid element_type  \
0         node/305063595  305063…

In [14]:
amenity = load_city_tag_raw(city, "amenity")
k_amenity = KeplerGl(data={"amenity": amenity})
k_amenity

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'amenity':                unique_id      osmid element_type           amenity  \
0         node…

In [15]:
amenity_counts = load_city_tag_counts(city, "amenity")
k_amenity_c = KeplerGl(data={"amenity": amenity_counts}, config=load_config("amenity"))
k_amenity_c

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'whyo7ar', 'type': …

In [16]:
waterway = load_city_tag_raw(city, "waterway")
k_waterway = KeplerGl(data={"waterway": waterway})
k_waterway

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(data={'waterway':              unique_id      osmid element_type  \
0       node/252954889  252954889…

In [17]:
waterway_counts = load_city_tag_counts(city, "waterway")
k_waterway_c = KeplerGl(data={"waterway": waterway_counts}, config=load_config("waterway"))
k_waterway_c

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'tw104h9h', 'type':…

In [18]:
leisure_counts = load_city_tag_counts(city, "leisure")
k_leisure_c = KeplerGl(data={"leisure": leisure_counts}, config=load_config("leisure"))
k_leisure_c

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


KeplerGl(config={'version': 'v1', 'config': {'visState': {'filters': [], 'layers': [{'id': 'en9x89', 'type': '…